In [1]:
import pandas as pd
import shapely
import os

import plotly.express as px

# Explore data

Terminology I need to define
* Division
* Booth
* AEC
* 2019 election information: why (booths change each election, diviosn borders change frequently as well)
* PPVC pre polling voting centre

In this document I want to:
* Explore current divisions and polling booths
* Plot them using plotly: have 2019 division as one region, and the booths shown inside. 
* Create a voronoi tesselation for the booths
* Plot the voronoi tesselation


## Get data
Data is available from the Australian Electoral Commision 

In [2]:
booth_info_loc = 'data/20190518/GeneralPollingPlacesDownload-24310.csv'

try:
    booths = pd.read_csv(booth_info_loc,
                         skiprows=1)
except IOError:
    booths = pd.read_csv('https://results.aec.gov.au/24310/Website/Downloads/GeneralPollingPlacesDownload-24310.csv')
    booths.to_csv(booth_info_loc)
    booths = pd.read_csv(booth_info_loc,
                         skiprows=1)

booths.head()

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
0,ACT,318,Bean,93925,5,Belconnen BEAN PPVC,Belconnen Community Centre,26 Chandler St,NaN,NaN,BELCONNEN,ACT,2900.0,-35.238930,149.069655
1,ACT,318,Bean,93927,5,BLV Bean PPVC,BLV Canberra,50 Marcus Clarke St,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.277334,149.126869
2,ACT,318,Bean,11877,1,Bonython,Bonython Primary School,64 Hurtle Ave,NaN,NaN,BONYTHON,ACT,2905.0,-35.431800,149.083000
3,ACT,318,Bean,11452,1,Calwell,Calwell High School,111 Casey Cres,NaN,NaN,CALWELL,ACT,2905.0,-35.440670,149.117600
4,ACT,318,Bean,8761,1,Chapman,Chapman Primary School,46-50 Perry Dr,NaN,NaN,CHAPMAN,ACT,2611.0,-35.356400,149.042000


## First just going to look at booths that I am familiar with

Testing with a small region to get experience with the data.

Data quality issues:
* Multiple booths with the same address but different lon/lat. I think it might be something with the polling place type id


In [3]:

booths2500 = booths[booths['PremisesPostCode']==2500]
booths2500

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
724,NSW,114,Cunningham,65460,5,BLV Cunningham PPVC,BLV Cunningham,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427135,150.898067
729,NSW,114,Cunningham,539,1,Coniston,Coniston Public School,123 Auburn St,NaN,NaN,CONISTON,NSW,2500.0,-34.438600,150.887000
735,NSW,114,Cunningham,30105,5,Divisional Office (PREPOLL),Divisional Office,Corporate Square,Level 4,43 Burelli St,WOLLONGONG,NSW,2500.0,-34.427011,150.898107
739,NSW,114,Cunningham,544,1,Gwynneville,Gwynneville Primary School,10A Acacia Ave,NaN,NaN,GWYNNEVILLE,NSW,2500.0,-34.418079,150.879339
742,NSW,114,Cunningham,546,1,Keiraville,Keiraville Public School,286 Gipps Rd,NaN,NaN,KEIRAVILLE,NSW,2500.0,-34.414700,150.873000
744,NSW,114,Cunningham,555,1,Mangerton,Mount St Thomas Public School,12-14 Taronga Ave,NaN,NaN,MANGERTON,NSW,2500.0,-34.437281,150.870466
745,NSW,114,Cunningham,553,1,Mount Keira,Edmund Rice College,112 Mount Keira Rd,NaN,NaN,WEST WOLLONGONG,NSW,2500.0,-34.419980,150.863017
765,NSW,114,Cunningham,565,1,Wollongong,Wollongong Town Hall,93 Crown St,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.425718,150.897710
766,NSW,114,Cunningham,34033,5,Wollongong CUNNINGHAM PPVC,3/51 Crown St,NaN,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.426057,150.899519
767,NSW,114,Cunningham,83535,1,Wollongong East,Wollongong Public School,67a Church St,NaN,NaN,WOLLONGONG,NSW,2500.0,-34.422548,150.896217


In [11]:
fig = px.scatter_mapbox(booths2500, lat="Latitude", lon="Longitude", hover_name="PremisesNm", 
                        hover_data=["DivisionNm", "PollingPlaceNm"],
                        zoom=11, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [16]:
booths.groupby('PollingPlaceTypeID')['DivisionID'].count()

PollingPlaceTypeID
1    7169
2     489
3      42
4      26
5    1149
Name: DivisionID, dtype: int64

In [17]:
booths[booths['PollingPlaceTypeID']==5]

,State,DivisionID,DivisionNm,PollingPlaceID,PollingPlaceTypeID,PollingPlaceNm,PremisesNm,PremisesAddress1,PremisesAddress2,PremisesAddress3,PremisesSuburb,PremisesStateAb,PremisesPostCode,Latitude,Longitude
0,ACT,318,Bean,93925,5,Belconnen BEAN PPVC,Belconnen Community Centre,26 Chandler St,NaN,NaN,BELCONNEN,ACT,2900.0,-35.238930,149.069655
1,ACT,318,Bean,93927,5,BLV Bean PPVC,BLV Canberra,50 Marcus Clarke St,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.277334,149.126869
7,ACT,318,Bean,93922,5,City BEAN PPVC,Canberra Museum and Gallery,176 London Crct,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.280517,149.130861
12,ACT,318,Bean,93932,5,Divisional Office (PREPOLL),Divisional Office,50 Marcus Clarke St,NaN,NaN,CANBERRA CITY,ACT,2601.0,-35.277334,149.126869
22,ACT,318,Bean,93929,5,Gungahlin BEAN PPVC,Quokka Pavilion,Flemington Rd,NaN,NaN,MITCHELL,ACT,2911.0,-35.219324,149.145173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8831,WA,248,Tangney,64592,5,Ardross PPVC,14 Riseley St,NaN,NaN,NaN,ARDROSS,WA,6153.0,-32.020883,115.834731
8837,WA,248,Tangney,65478,5,BLV Tangney PPVC,BLV Tangney,Level 13,200 St Georges Tce,NaN,PERTH,WA,6000.0,-31.953563,115.853295
8843,WA,248,Tangney,83332,5,Cannington TANGNEY PPVC,1280 ALbany Hwy,NaN,NaN,NaN,CANNINGTON,WA,6107.0,-32.017000,115.934000
8844,WA,248,Tangney,30239,5,Divisional Office (PREPOLL),Divisional Office,Level 13,200 St Georges Tce,NaN,PERTH,WA,6000.0,-31.952853,115.853352
